## <strong > Importation de modules </strong>

In [3]:
from sklearnex import patch_sklearn
patch_sklearn()

############################################

import numpy as np
import pandas as pd
from scipy import stats
import matplotlib.pyplot as plt
import sklearn

ImportError: cannot import name 'sparse_lsqr' from 'sklearn.utils.fixes' (/home/brice_kengni_zanguim/anaconda3/envs/OC-Projects/lib/python3.10/site-packages/sklearn/utils/fixes.py)

## <strong> Mes fonctions </strong>

In [4]:
def rayon_de_la_terre (latitude , unit = "rad" , raise_error = False):
    """
        La fonction permet l'évaluation du rayon de la terre en fonction de la lattitude à laquelle on se situe à la surface
        L'éqation utilisée est expliquée et décrite sur le site suivant : https://planetcalc.com/7721/
    
    paramètres :
    -------------
    latitude : float
            latitude du point
    unit : str
            Unité dans laquelle s'exprime la latitude. les valeurs permises sont "deg" et "rad"
    raise_error : Bool
                Variable Booléene pour préciser si l'utilisateur lever une erreur lorsque la variable 'unit' est mal renseignée ou fournie
        
    Return :
    ---------
        Rayon : float
                Raoyn de la terre à la latitude fournie en entrée.
    """
    if unit == "deg" :
        latitude = latitude*np.pi/180
    elif unit == "rad" :
        latitude = latitude
    else :
        if raise_error :
            raise  f"Les seules veleurs admises pour la variable 'unit' sont 'deg' et 'rad'. La valeur que vous avez fournie est '{unit}'"
        else :
            return f"Les seules veleurs admises pour la variable 'unit' sont 'deg' et 'rad'. La valeur que vous avez fournie est '{unit}'"
        
    R_1 = 6378.137  # Rayon au niveau de la mer à l'équateur en kilomètres
    R_2 = 6356.752  # Rayon de la terre aux poles en, Kilomètres

    c = (R_1**2*np.cos(latitude))**2
    d = (R_2**2*np.sin(latitude))**2
    e = (R_1*np.cos(latitude))**2
    f = (R_2*np.sin(latitude))**2
    return np.sqrt((c+d)/(e+f))

def distance_a_vol_oiseau ( teta_a , teta_b , phi_a , phi_b , n = 30 ):
    """
        La fonction effectue le calcul de la distance à vol d'oiseau entre deux position . 
        Le calcul de la distance est faite en approximant une intégrale par la méthode des trapèze
    
    Paramètres :
    -------------
        teta_a : float
            borne inférieure de la latitude
        teta_b : float
            borne superieure de la latitude
        phi_a : float
            borne inférieure de la longitude
        phi_b : float
            borne superieure de la longitude
        n  : int
            nombre de divisions de l'intervale [a,b]. Le nombre de discretisation est alors n+1
    
    Return :
    ---------
        integrale : float
    """
    phi_b , phi_a = abs(phi_b - phi_a)%180 , 0
    #  conversion des angles en radian
    teta_a , teta_b , phi_a , phi_b =  teta_a*np.pi/180 , teta_b*np.pi/180 , phi_a*np.pi/180 , phi_b*np.pi/180
    
    teta =  np.linspace( teta_a, teta_b, n+1 )
    
    if np.nan in [teta_a , teta_b , phi_a , phi_b] : 
        return np.nan
    
    if phi_a == phi_b :
        if teta_a == teta_b :
            return 0.0
        else :
            integrant = rayon_de_la_terre( teta )
            h = (teta[-1] - teta[0])/n
    else :
        phi =  np.linspace( phi_a, phi_b, n+1 )
        a = (teta_a - teta_b)/(phi_a - phi_b)
        integrant = rayon_de_la_terre( teta )*np.sqrt( a**2 + np.sin(teta)**2 )
        h = (phi[-1] - phi[0])/n
    
    return  abs(h)*( 0.5*( integrant[0] + integrant[-1] )  +  integrant[1:-1].sum() )


def f(x):
    """ moyenne de jours qui s'écroule entre deux achats consécutifs d'un client"""
    if x.shape[0] == 1 :
        return 0
    else :
        x = x.values
        x = np.array( [ pd.to_datetime( x[i] , yearfirst=True )  for i in range(len(x)) ] )
        x = np.array([  ( x[i+1] - x[i] ).total_seconds() for i in range(len(x)-1)])
        return (x).mean()/(24.*3600.)

def series_modes (x) :
    if x.shape[0] ==0 :
        return np.nan
    else :
        return stats.mode(x)[0][0]
    
def couleur_léatoire_hex() :
    """
    Générateur de couleur aléatoire dans le format hexadecimal

    paramètres :  
    ------------
        Aucun paramètre à fournir

    Returns : 
    ---------
        String : Code hexadecimal de la couleur fabriquée sous forme d'une variable chaine de caractères.
    """
    s = "#"
    a = np.random.choice(["0","1","2","3","4","5","6","7","8","9","A","B","C","D","E","F"] , 6 ,replace=False)
    for i in a : s += i
    return s

def camemberg ( labels = ["Brice","Romeo","Marthe","Franck","Arnaud"] , sizes=[28,18,23,30,15] ,titre = "CAMENBERG", figure_size =(8,8) , rot = 30 ) :
    """
    La fonction permet d'afficher un camemberg selon les entrée fournies
    
    Paramètre :
    -----------
        labels : Array_like 
                Labels des données à afficher sous forme de camemberg
        sizes :  Array_like 
                Grandeurs quantitatives correspondants aux labels fournis
        titre : str
                Titre du graphique
        figure_size : tuple
                respectivement la largeur et la hauteur du graphique sous forme de tuple
        
    Return :
    --------
        None
    """
    # Data to plot
    label = np.array(labels)
    size = np.array(sizes)
    colors = [couleur_léatoire_hex() for i in range(size.size) ]
    explode = np.array([ 0. for i in range(size.size) ])  # explode 1st slice
    explode[size == size.max()] = 0.08

    # Plot
    plt.figure( figsize = figure_size )
    plt.title(titre, color ='brown' , size = 18 )
    plt.pie(size, explode=explode, labels=label, colors=colors, autopct='%1.1f%%', shadow=True, startangle=rot, textprops={'fontsize': figure_size[0]*2})
    plt.axis('equal')
    plt.show()
    

def selection_colonnes_norm (data , columns = ["Nombre_commandes","Jour_moyen_entre_achats","payment_value"]):
    
    if "review_score" in columns :
        columns.remove("review_score")
        base = data[columns]
        base_norm = pd.DataFrame( sklearn.preprocessing.scale(base) , columns= columns )
        base_norm["review_score"] = data["review_score"]
    else :
        base = data[columns]
        base_norm = pd.DataFrame( sklearn.preprocessing.scale(base) , columns= columns )
    return base_norm


def display_circles(pcs, n_comp, pca, axis_ranks, labels=None, label_rotation=0,
                        lims=None , fgsize = (12,12),label_num=False,labl_arrow_size = 14):
    for d1, d2 in axis_ranks: # On affiche les 3 premiers plans factoriels, donc les 6 premières composantes
        if d2 < n_comp:

            # initialisation de la figure
            fig, ax = plt.subplots(figsize=fgsize)

            # détermination des limites du graphique
            if lims is not None :
                xmin, xmax, ymin, ymax = lims
            elif pcs.shape[1] < 30 :
                xmin, xmax, ymin, ymax = -1, 1, -1, 1
            else :
                xmin, xmax, ymin, ymax = min(pcs[d1,:]), max(pcs[d1,:]), min(pcs[d2,:]), max(pcs[d2,:])

            # affichage des flèches
            # s'il y a plus de 30 flèches, on n'affiche pas le triangle à leur extrémité
            vsbl = 0.3
            if pcs.shape[1] < 30 :
                plt.quiver(np.zeros(pcs.shape[1]), np.zeros(pcs.shape[1]), 
                   pcs[d1,:], pcs[d2,:],  angles='xy', scale_units='xy', scale=1, color="green",width = 0.005, alpha=vsbl)
                # (voir la doc : https://matplotlib.org/api/_as_gen/matplotlib.pyplot.quiver.html)
            else:
                lines = [[[0,0],[x,y]] for x,y in pcs[[d1,d2]].T]
                ax.add_collection(LineCollection(lines, axes=ax, alpha=vsbl, color='black'))
            
            # affichage des noms des variables  
            if labels is not None:  
                for i,(x, y) in enumerate(pcs[[d1,d2]].T):
                    fluct = np.random.normal(0,0.02,x.shape)
                    if x >= xmin and x <= xmax and y >= ymin and y <= ymax :
                        if label_num :
                            plt.text(x+fluct, y+fluct, f"{i}", fontsize=labl_arrow_size, ha='center', va='center', 
                                                            rotation=label_rotation, color="magenta", alpha=vsbl)
                        else :
                            plt.text(x+fluct, y+fluct, labels[i], fontsize=labl_arrow_size, ha='center', va='center', 
                                                            rotation=label_rotation, color="magenta", alpha=vsbl)
            
            # affichage du cercle
            circle = plt.Circle((0,0), 1, facecolor='none', edgecolor='green')
            plt.gca().add_artist(circle)

            # définition des limites du graphique
            plt.xlim(xmin, xmax)
            plt.ylim(ymin, ymax)
        
            # affichage des lignes horizontales et verticales
            plt.plot([-1, 1], [0, 0], color='grey', ls='--')
            plt.plot([0, 0], [-1, 1], color='grey', ls='--')

            # nom des axes, avec le pourcentage d'inertie expliqué
            plt.xlabel('F{} ({}%)'.format(d1+1, round(100*pca.explained_variance_ratio_[d1],1)))
            plt.ylabel('F{} ({}%)'.format(d2+1, round(100*pca.explained_variance_ratio_[d2],1)))

            plt.title(f"Cercle des corrélations - directions principales F{d1+1} et F{d2+1}", size=15)
            plt.show(block=False)
        
def display_factorial_planes(X_projected, n_comp, pca, axis_ranks, labels=None, alpha=0.6, 
                                          illustrative_var=None,show_label_element = True , fgsize = (12,8)):
    for d1,d2 in axis_ranks:
        if d2 < n_comp:
            # initialisation de la figure       
            fig = plt.figure(figsize=fgsize)
        
            # affichage des points
            if illustrative_var is None:
                plt.scatter(X_projected[:, d1], X_projected[:, d2], alpha=alpha, c = labels)
            else:
                illustrative_var = np.array(illustrative_var)
                for value in np.unique(illustrative_var):
                    selected = np.where(illustrative_var == value)
                    plt.scatter(X_projected[selected, d1], X_projected[selected, d2], alpha=alpha, label=value, c = labels)
                plt.legend()

            # affichage des labels des points
            if (labels is not None)&(show_label_element):
                for i,(x,y) in enumerate(X_projected[:,[d1,d2]]):
                    plt.text(x, y, labels[i], fontsize='14', ha='center',va='center', alpha=0.5) 
                
            # détermination des limites du graphique
            
            plt.xlim([np.min(X_projected[:, [d1]]) * 1.1  ,  np.max(X_projected[:, [d1]]) * 1.1])
            plt.ylim([np.min(X_projected[:, [d2]]) * 1.1  ,  np.max(X_projected[:, [d2]]) * 1.1])
        
            # affichage des lignes horizontales et verticales
            plt.plot([-100, 100], [0, 0], color='grey', ls='--')
            plt.plot([0, 0], [-100, 100], color='grey', ls='--')

            # nom des axes, avec le pourcentage d'inertie expliqué
            plt.xlabel('F{} ({}%)'.format(d1+1, round(100*pca.explained_variance_ratio_[d1],1)))
            plt.ylabel('F{} ({}%)'.format(d2+1, round(100*pca.explained_variance_ratio_[d2],1)))

            plt.title(f"Projection des individus- directions principales F{d1+1} et F{d2+1}",size=15)
            plt.show(block=False)

def display_scree_plot(pca , fgsize = (12,8)):
    scree = pca.explained_variance_ratio_*100
    plt.figure(figsize= fgsize ) 
    plt.bar(np.arange(len(scree))+1, scree)
    plt.plot(np.arange(len(scree))+1, scree.cumsum(),c="red",marker='o')
    plt.xlabel("rang de l'axe d'inertie")
    plt.ylabel("pourcentage d'inertie")
    plt.title("Eboulis des valeurs propres")
    plt.show(block=False)

